In [1]:
import pandas as pd
from dataPreparation import data_preparation

df_rent, df_house_price, df_rent_price = data_preparation()

# there are NaN, we replace them with 0
df = df_rent.fillna(0)
df_house_price = df_house_price.fillna(0)
df_rent_price = df_rent_price.fillna(0)

df

,id,title,textdata,desktophidden2,desktophidden4
0,01G6N8XD1YS535N57X5KXTH4W1,"Apartment for rent in Bucureşti - Sectorul 4,...",56 m2 apartment for rent in Bucureşti - Sector...,650 RON,56 m2
1,01G6N8WK3NWXJ98NJQ8KF4Y050,"Apartment for rent in Bad Vöslau, Austria",77 m2 apartment for rent in Bad Vöslau,749 EUR,77 m2
2,01G6N8X988HGEN112QYR0XJC7M,"Apartment for rent in Sliema, Malta",210 m2 apartment for rent in Sliema,"5,700 EUR",210 m2
3,01G6N8WC328DDNW0QM82JXKEP7,"Apartment for rent in Polla, Italy",90 m2 apartment for rent in Polla,"2,400 EUR",90 m2
4,01G6N8X2XXHT8DTWN7837MXATN,"Apartment for rent in Prague 2, Czech Republic",56 m2 apartment for rent in Prague 2,"23,000 CZK",56 m2
...,...,...,...,...,...
6472,01G6N8WP2WBMZHNEYFS5ZJKA4Y,"Apartment for rent in Hinterland, Switzerland",Apartment for rent in Hinterland,Not specified,0
6473,01G6N8WWBYY0HVB2CVDENWKQ7T,"Apartment for rent in Warszawa Ursynów, Poland",91 m2 apartment for rent in Warszawa Ursynów,"6,000 PLN",91 m2
6474,01G6N8WV11T86AJA9GB9Z1FSCC,"Apartment for rent in Risskov, Denmark",92 m2 apartment for rent in Risskov,"11,900 DKK",92 m2
6475,01G6N8WERJSF7HHT3P8MYTFKXQ,"Apartment for rent in Leganés, Spain",90 m2 apartment for rent in Leganés,990 EUR,90 m2


In [2]:
df.rename(columns = {'textdata':'text-data', 'desktophidden2':'desktop-hidden 2', 'desktophidden4':'desktop-hidden 4',}, inplace = True)

# create a new column with the country name by taking the last word of the column 'title'

splitted = df['title'].str.split()
df['country'] = splitted.str[-1]
df['country'].replace({'Republic':'Czech Republic'}) # replace the 'Republic' with 'Czech Republic'

# drop useless columns
df.drop(columns=['title'], inplace=True)
df.drop(columns=['text-data'], inplace=True)

df

,id,desktop-hidden 2,desktop-hidden 4,country
0,01G6N8XD1YS535N57X5KXTH4W1,650 RON,56 m2,Romania
1,01G6N8WK3NWXJ98NJQ8KF4Y050,749 EUR,77 m2,Austria
2,01G6N8X988HGEN112QYR0XJC7M,"5,700 EUR",210 m2,Malta
3,01G6N8WC328DDNW0QM82JXKEP7,"2,400 EUR",90 m2,Italy
4,01G6N8X2XXHT8DTWN7837MXATN,"23,000 CZK",56 m2,Republic
...,...,...,...,...
6472,01G6N8WP2WBMZHNEYFS5ZJKA4Y,Not specified,0,Switzerland
6473,01G6N8WWBYY0HVB2CVDENWKQ7T,"6,000 PLN",91 m2,Poland
6474,01G6N8WV11T86AJA9GB9Z1FSCC,"11,900 DKK",92 m2,Denmark
6475,01G6N8WERJSF7HHT3P8MYTFKXQ,990 EUR,90 m2,Spain


In [3]:
# rename the columns and replace 'Not specified' prices with 0
df.rename(columns={"desktop-hidden 2": "price", "desktop-hidden 4": "surface"}, inplace=True)

df['price'] = df['price'].astype(str)
df['price'].replace({'0,':'0'}, regex=True, inplace=True)
df['price'].replace({'1,':'1'}, regex=True, inplace=True)
df['price'].replace({'2,':'2'}, regex=True, inplace=True)
df['price'].replace({'3,':'3'}, regex=True, inplace=True)
df['price'].replace({'4,':'4'}, regex=True, inplace=True)
df['price'].replace({'5,':'5'}, regex=True, inplace=True)
df['price'].replace({'6,':'6'}, regex=True, inplace=True)
df['price'].replace({'7,':'7'}, regex=True, inplace=True)
df['price'].replace({'8,':'8'}, regex=True, inplace=True)
df['price'].replace({'9,':'9'}, regex=True, inplace=True)

df['surface'].replace({'0,':'0'}, regex=True, inplace=True)
df['surface'].replace({'1,':'1'}, regex=True, inplace=True)
df['surface'].replace({'2,':'2'}, regex=True, inplace=True)
df['surface'].replace({'3,':'3'}, regex=True, inplace=True)
df['surface'].replace({'4,':'4'}, regex=True, inplace=True)
df['surface'].replace({'5,':'5'}, regex=True, inplace=True)
df['surface'].replace({'6,':'6'}, regex=True, inplace=True)
df['surface'].replace({'7,':'7'}, regex=True, inplace=True)
df['surface'].replace({'8,':'8'}, regex=True, inplace=True)
df['surface'].replace({'9,':'9'}, regex=True, inplace=True)

df = df[df['price'] != "Not specified"]
df = df[df['surface'] != 0]
df = df[df['price'] != 0]

df

,id,price,surface,country
0,01G6N8XD1YS535N57X5KXTH4W1,650 RON,56 m2,Romania
1,01G6N8WK3NWXJ98NJQ8KF4Y050,749 EUR,77 m2,Austria
2,01G6N8X988HGEN112QYR0XJC7M,5700 EUR,210 m2,Malta
3,01G6N8WC328DDNW0QM82JXKEP7,2400 EUR,90 m2,Italy
4,01G6N8X2XXHT8DTWN7837MXATN,23000 CZK,56 m2,Republic
...,...,...,...,...
6471,01G6N8X3AQCM5HNVXF9F4S8QWG,15000 CZK,72 m2,Republic
6473,01G6N8WWBYY0HVB2CVDENWKQ7T,6000 PLN,91 m2,Poland
6474,01G6N8WV11T86AJA9GB9Z1FSCC,11900 DKK,92 m2,Denmark
6475,01G6N8WERJSF7HHT3P8MYTFKXQ,990 EUR,90 m2,Spain


In [4]:
#convert all the prices to € and remove the 'EUR'

# CHF, DKK, PLN, SEK, CZK, BGN, HUF, RON, Iceland
# create multiplier for the prices
multiplier = {'EUR': 1, 'CHF': 1, 'DKK': 0.13, 'PLN': 0.21, 'SEK': 0.09, 'CZK': 0.04, 'BGN': 0.51, 'HUF': 0.0025, 'RON': 0.2, 'ISK': 0.0072, 'NOK': 0.096}

prices = []
for price in df['price']:
    if price.find('EUR') != -1:
        price = price.replace('EUR', '')
        price = float(price) * multiplier['EUR']
    elif price.find('CHF') != -1:
        price = price.replace('CHF', '')
        price = float(price) * multiplier['CHF']
    elif price.find('DKK') != -1:
        price = price.replace('DKK', '')
        price = float(price) * multiplier['DKK']
    elif price.find('PLN') != -1:
        price = price.replace('PLN', '')
        price = float(price) * multiplier['PLN']
    elif price.find('SEK') != -1:
        price = price.replace('SEK', '')
        price = float(price) * multiplier['SEK']
    elif price.find('CZK') != -1:
        price = price.replace('CZK', '')
        price = float(price) * multiplier['CZK']
    elif price.find('BGN') != -1:
        price = price.replace('BGN', '')
        price = float(price) * multiplier['BGN']
    elif price.find('HUF') != -1:
        price = price.replace('HUF', '')
        price = float(price) * multiplier['HUF']
    elif price.find('RON') != -1:
        price = price.replace('RON', '')
        price = float(price) * multiplier['RON']
    elif price.find('ISK') != -1:
        price = price.replace('ISK', '')
        price = float(price) * multiplier['ISK']
    elif price.find('NOK') != -1:
        price = price.replace('NOK', '')
        price = float(price) * multiplier['NOK']
    else:
        price = float(price)
    prices.append(price)
df['price'] = prices

#mult = df['price'].str.contains('PLN').map({ True : 0.21, False : 1})
#mult = df['price'].str.contains('SEK').map({ True : 0.09, False : 1})
#mult = df['price'].str.contains('CZK').map({ True : 0.04, False : 1})
#mult = df['price'].str.contains('BGN').map({ True : 0.51, False : 1})
#mult = df['price'].str.contains('HUF').map({ True : 0.0025, False : 1})
#mult = df['price'].str.contains('RON').map({ True : 0.2, False : 1})
# extract values
#values = df['price'].str.replace(r'\D+', '').astype(float)
#df['price'] = values * mult
#df['price'] = df['price'].astype(int)

print(df)

                              id   price surface   country
0     01G6N8XD1YS535N57X5KXTH4W1   130.0   56 m2   Romania
1     01G6N8WK3NWXJ98NJQ8KF4Y050   749.0   77 m2   Austria
2     01G6N8X988HGEN112QYR0XJC7M  5700.0  210 m2     Malta
3     01G6N8WC328DDNW0QM82JXKEP7  2400.0   90 m2     Italy
4     01G6N8X2XXHT8DTWN7837MXATN   920.0   56 m2  Republic
...                          ...     ...     ...       ...
6471  01G6N8X3AQCM5HNVXF9F4S8QWG   600.0   72 m2  Republic
6473  01G6N8WWBYY0HVB2CVDENWKQ7T  1260.0   91 m2    Poland
6474  01G6N8WV11T86AJA9GB9Z1FSCC  1547.0   92 m2   Denmark
6475  01G6N8WERJSF7HHT3P8MYTFKXQ   990.0   90 m2     Spain
6476  01G6N8WW0YAXZ2KDBRGCQDPB0R   525.0   35 m2    Poland

[5043 rows x 4 columns]


In [ ]:
df['surface'].replace({' m2':''}, regex=True, inplace=True)
df['surface'] = df['surface'].astype(float)

df['index'] = df['price']/df['surface']
# remove ousiders
df = df[df['index'] < 1000]

df.to_csv('result.csv', sep='\t')

In [ ]:
#df = df[df['index'] < 100]
df = df[df['index'].isna() == False]
df.groupby(['country'])['index'].mean().plot(legend=True, kind='bar', figsize=(10,5), title='Average index by country', xlabel='Country', ylabel='Average index')

In [ ]:
df.groupby(['country'])['price'].mean().plot(legend=True, kind='bar', figsize=(10,5))

In [ ]:
# Create a Map with geojson

In [ ]:
df_house_price.rename(columns={"Unnamed: 0": "country"}, inplace=True)
df_house_price

In [ ]:
import matplotlib.pyplot as plt

df_rent_price.rename(columns={"Unnamed: 0": "country"}, inplace=True)
df_rent_price

In [ ]:
ax = plt.gca()
  
#use plot() method on the dataframe
df_rent_price.plot( x = 'country', y = '2020', ax = ax, figsize=(10,5), legend=True, kind='bar', color='red', rot=0)
df_house_price.plot( x = 'country', y = '2020', ax = ax, figsize=(10,5), legend=True, kind='bar', color='blue')

In [ ]:

countries = df_rent_price['country']
rentSeries = df_rent_price['2020']
priceSeries = df_house_price['2020']

df = pd.DataFrame({"Rent":rentSeries,"Price":priceSeries})

ax = df.plot.bar(color=["SkyBlue","IndianRed"], rot=0, title="Index complare\nRent vs. Price", figsize=(10,5))
ax.set_xlabel("country")
ax.set_ylabel("indexes")
plt.show()

In [ ]:
ax = plt.gca()
  
#use plot() method on the dataframe
df_house_price.plot( x = 'country', y = '2020', ax = ax, figsize=(10,5), legend=True, kind='bar')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

PRICE_LENGTH = df["price"].values
INDEX_LENGTH = df["index"].values

COUNTRIES = df["country"].values
COUNTRIES_ = np.unique(COUNTRIES)

COLORS = ["#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02"]

fig, ax = plt.subplots(figsize=(8,8))
for country, color in zip(COUNTRIES_, COLORS):
    idxs = np.where(COUNTRIES == country)
    # No legend will be generated if we don't pass label=species
    ax.scatter(
        PRICE_LENGTH[idxs], INDEX_LENGTH[idxs], label=country,
        s=50, color=color, alpha=0.7
    )
    
ax.legend();